In [ ]:
### Neural collaborative filtering###

In [ ]:
### data processing ###

## we work, first, with a reduced size version of MovieLens... ##

## importing datasets
from google.colab import files
uploaded = files.upload()

Saving ratings.csv to ratings.csv
Saving movies.csv to movies.csv


In [ ]:
import pandas as pd

import torch 
import torch.nn as nn
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
ratings_df = pd.read_csv('ratings.csv')
movies_df = pd.read_csv('movies.csv')

TOP_K = 5
N_EPOCHS = 10

In [ ]:
## data preprocessing... ##
# we want to simlate implicit feedback because in real life we almost always have to deal with it #

ratings_df["rating"] = (ratings_df["rating"]>=3).astype("int8")
ratings_df=ratings_df.drop(["timestamp"], axis=1)
l = list(np.unique(ratings_df["movieId"]))
l2 = list(np.unique(ratings_df["userId"]))
ratings_df["movieId"]=ratings_df["movieId"].apply(lambda x: l.index(x))
ratings_df["userId"]=ratings_df["userId"].apply(lambda x: l2.index(x))

R_ij= ratings_df.pivot(index = 'userId', columns ='movieId', values = 'rating').fillna(0)

In [ ]:
R_ij

movieId,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,9026,9027,9028,9029,9030,9031,9032,9033,9034,9035,9036,9037,9038,9039,9040,9041,9042,9043,9044,9045,9046,9047,9048,9049,9050,9051,9052,9053,9054,9055,9056,9057,9058,9059,9060,9061,9062,9063,9064,9065
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
666,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
668,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
from typing import List


def wide_to_long(wide: np.array, possible_ratings: List[int]) -> np.array:
    """Go from wide table to long.
    :param wide: wide array with user-item interactions
    :param possible_ratings: list of possible ratings that we may have."""

    def _get_ratings(arr: np.array, rating: int) -> np.array:
        """Generate long array for the rating provided
        :param arr: wide array with user-item interactions
        :param rating: the rating that we are interested"""
        idx = np.where(arr == rating)
        return np.vstack(
            (idx[0], idx[1], np.ones(idx[0].size, dtype="int8") * rating)
        ).T

    long_arrays = []
    for r in possible_ratings:
        long_arrays.append(_get_ratings(wide, r))

    return np.vstack(long_arrays)

In [ ]:
unique_ratings = np.unique(R_ij)
long_train = wide_to_long(R_ij, unique_ratings)
df_train = pd.DataFrame(long_train, columns=["userId", "movieId", "rating"])

In [ ]:
df_test=df_train.copy()

In [ ]:
class RNN(nn.Module):
    def __init__(self, nb_users, nb_items):
        super(RNN, self).__init__()
        self.mlp_embedding_user = nn.Embedding(nb_users,32)
        self.mlp_embedding_item = nn.Embedding(nb_items,32)
        self.mtx_embedding_user = nn.Embedding(nb_users,4)
        self.mtx_embedding_item = nn.Embedding(nb_items,4)
        self.fc1 = nn.Linear(64, 8)
        self.fc3 = nn.Linear(8, 4)
        self.out= nn.Linear(8,1)
        self.relu=nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        
    def forward(self,input):
        user =input[:, 0].view((-1,1))
        item = input[:, 1].view((-1,1))
        eu1 = self.mlp_embedding_user(user)
        ei1 = self.mlp_embedding_item(item)
        combined = torch.cat((eu1, ei1), 1).flatten(start_dim=1)
        combined= self.fc1(combined)
        combined = self.relu(combined)
        combined= self.fc3(combined)
        combined = self.relu(combined)
        eu2 = self.mtx_embedding_user(user).flatten(start_dim=1)
        ei2 = self.mtx_embedding_item(item).flatten(start_dim=1)
        prod =ei2*eu2
        combined = torch.cat((combined, prod), 1).flatten(start_dim=1)
        out = self.sigmoid(self.out(combined))
        return out




In [ ]:
def train_step_constructor(model,loss_f,optimizer):
  def train_step(batch_x, batch_y):
    p = model(batch_x)
    loss = loss_f( p.view((-1,)),batch_y.view((-1,)))
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    return loss.item()
  return train_step

In [ ]:
from torch.utils.data import Dataset

class The_Dataset(Dataset):
    def __init__(self, x_arr, y_arr):
        ## we can here connect to our database
        self.x = torch.tensor(x_arr)
        self.y = torch.tensor(y_arr)
        
    def __getitem__(self, index):
        return (self.x[index], self.y[index])

 

    def __len__(self):
        return len(self.x)

In [ ]:
ratings_df.describe()

In [ ]:
train,test = train_test_split(df_train,test_size=0.33, random_state=42)

In [ ]:
train_data = The_Dataset(np.array(train[["userId","movieId"]]), np.array(train["rating"]))

In [ ]:
from torch.utils.data import DataLoader


train_data = The_Dataset(np.array(train[["userId","movieId"]]), np.array(train["rating"]))
train_loader = DataLoader(dataset=train_data, batch_size=500, shuffle=True)

In [ ]:
R_ij.shape[1]

9066

In [ ]:
model =RNN(R_ij.shape[0],R_ij.shape[1])
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)
loss = nn.BCELoss()
step = train_step_constructor(model,loss,optimizer)

losses = []
val_losses = []

for epoch in range(50):
    model.train()
    cl=0
    for batch_x,batch_y in train_loader:
      ll=step(batch_x.long(), batch_y.float().detach())
      cl=cl+ll
      losses.append(ll)
    print(f"{epoch} : {cl}")
    if not epoch % 10 :
      torch.save(model.state_dict(), "/content/model")

0 : 577.6469918265939
1 : 379.77793937548995
2 : 370.5184697294608
3 : 365.1880166316405
4 : 360.1143687348813
5 : 354.60349233914167
6 : 348.187325255014
7 : 341.3397630872205
8 : 334.5102117843926
9 : 327.6218300247565
10 : 320.9588047484867
11 : 314.554310452193
12 : 308.25707936799154
13 : 302.57473181560636
14 : 297.1732151261531
15 : 292.32793447887525
16 : 287.81191397923976
17 : 283.9064082913101
18 : 280.41720887552947
19 : 277.2276552482508
20 : 274.52696966100484
21 : 271.959611131344
22 : 269.93591031990945
23 : 267.75775954453275
24 : 265.9178307563998
25 : 264.34959956258535
26 : 262.90768267493695
27 : 261.5020790286362
28 : 260.38658358482644
29 : 259.13838753988966
30 : 258.130838454701
31 : 257.34995262976736
32 : 256.50016686553136
33 : 255.72184689436108
34 : 254.97224356560037
35 : 254.35234774835408
36 : 253.79145793896168
37 : 253.08323833951727
38 : 252.6409787866287
39 : 252.10134901106358
40 : 251.64054454490542
41 : 251.1524895457551
42 : 250.95319093740545
4

In [ ]:
model =RNN(R_ij.shape[0],R_ij.shape[1])
model.load_state_dict(torch.load("/content/model"))

<All keys matched successfully>

In [ ]:
df_test=df_train.copy()
predictions=df_test.copy()
df_test=np.array(df_test[["userId","movieId"]])
df_test=torch.tensor(df_test).detach()

In [ ]:
model.eval()
predictions["rating"]=pd.DataFrame(model(df_test.long()).detach().numpy())

In [ ]:
pred = predictions.pivot(index="userId", columns="movieId", values="rating").fillna(0)
pred

movieId,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0,28.0,29.0,30.0,31.0,32.0,33.0,34.0,35.0,36.0,37.0,38.0,39.0,...,9026.0,9027.0,9028.0,9029.0,9030.0,9031.0,9032.0,9033.0,9034.0,9035.0,9036.0,9037.0,9038.0,9039.0,9040.0,9041.0,9042.0,9043.0,9044.0,9045.0,9046.0,9047.0,9048.0,9049.0,9050.0,9051.0,9052.0,9053.0,9054.0,9055.0,9056.0,9057.0,9058.0,9059.0,9060.0,9061.0,9062.0,9063.0,9064.0,9065.0
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0.0,0.012281,0.000676,0.000320,0.000026,0.000166,0.003639,0.000285,4.709539e-07,0.000101,0.011689,0.011593,0.000027,0.000014,0.000360,0.000003,0.003015,0.005369,0.000067,0.004740,0.000009,0.005849,0.001197,0.000242,0.002547,0.010333,0.000032,0.000017,0.001163,0.001646,0.000016,0.000349,0.030470,0.005657,0.000023,0.008446,0.000004,2.070247e-07,0.001616,0.000009,0.000436,...,1.164037e-05,6.307915e-07,9.016077e-07,8.642676e-06,1.172275e-07,0.000003,3.287822e-08,9.320052e-06,9.072157e-06,3.747792e-06,1.539173e-07,0.000008,0.000003,6.489292e-06,1.197581e-06,6.128563e-07,3.224338e-07,2.685632e-06,0.000002,3.364630e-06,2.803149e-06,5.764197e-06,8.212958e-05,1.406486e-08,4.965249e-07,5.592005e-08,0.000003,8.370713e-07,6.089000e-08,1.205826e-06,1.006321e-05,9.511051e-08,7.498022e-07,6.517485e-08,9.507054e-06,3.301553e-05,1.472854e-07,1.793663e-06,0.000002,1.500647e-06
1.0,0.330460,0.485667,0.060978,0.020467,0.144863,0.324586,0.066686,5.437573e-03,0.015483,0.640859,0.669102,0.013195,0.006503,0.038770,0.003765,0.046349,0.484446,0.008361,0.125852,0.012313,0.577803,0.093555,0.016635,0.016198,0.316422,0.001440,0.017044,0.042355,0.014832,0.006988,0.115051,0.457277,0.782452,0.008505,0.336800,0.000046,1.212525e-05,0.719187,0.000837,0.035362,...,4.847935e-10,3.205225e-05,1.879042e-08,4.678399e-10,8.250164e-05,0.000039,7.143665e-05,1.582812e-07,4.986404e-10,5.801134e-10,2.029336e-04,0.000129,0.000027,1.043060e-07,2.515455e-07,1.327256e-05,1.584310e-06,2.139406e-06,0.000063,1.640295e-09,7.232597e-10,1.826380e-10,2.961238e-08,4.171385e-05,2.010774e-05,7.253188e-08,0.000005,1.173345e-04,4.442831e-07,1.003898e-09,4.360108e-10,8.389920e-08,1.941558e-04,1.987044e-04,3.484857e-10,9.137408e-09,3.738608e-05,5.591414e-07,0.000995,4.396660e-10
2.0,0.306855,0.299306,0.019643,0.003047,0.007944,0.040699,0.001785,2.247168e-04,0.006238,0.074106,0.023766,0.007902,0.002625,0.004493,0.001835,0.154130,0.008078,0.053481,0.127787,0.008308,0.046785,0.031540,0.007854,0.050438,0.025053,0.000582,0.001439,0.000489,0.018357,0.001452,0.072691,0.174410,0.101181,0.000873,0.061645,0.000186,5.929850e-05,0.144407,0.001567,0.004680,...,1.339607e-07,4.045692e-05,1.548730e-06,1.384176e-07,8.610245e-04,0.000285,3.273353e-05,1.143618e-05,6.057458e-08,1.455135e-07,3.768798e-04,0.000457,0.000499,2.472925e-05,2.307535e-05,5.037781e-05,4.791247e-05,2.306261e-07,0.000656,8.908877e-07,1.510741e-07,6.549074e-08,3.342639e-05,1.201454e-05,2.020832e-05,1.803569e-06,0.000585,2.903687e-04,1.247987e-05,1.638181e-07,1.812326e-07,1.302544e-06,1.593761e-04,1.439747e-04,4.515486e-08,5.366039e-06,2.626528e-05,1.901731e-05,0.000403,1.755147e-07
3.0,0.443781,0.106031,0.138329,0.006777,0.101401,0.116220,0.295688,3.347170e-03,0.053756,0.340268,0.159807,0.008493,0.004251,0.009057,0.000460,0.045274,0.095712,0.025492,0.057685,0.001552,0.129172,0.038951,0.005716,0.110868,0.086661,0.002250,0.002755,0.007915,0.039748,0.002153,0.019821,0.358000,0.450557,0.004719,0.121168,0.000374,2.269446e-05,0.238054,0.005539,0.007657,...,1.551943e-06,8.289413e-05,1.499967e-06,1.552334e-06,5.585290e-05,0.000731,1.670274e-05,2.075297e-05,1.789583e-06,1.414553e-06,3.587738e-05,0.000508,0.000360,1.806429e-05,2.696516e-05,3.758719e-04,4.696801e-05,1.823246e-06,0.000088,4.070723e-06,1.490965e-06,1.002862e-06,1.005053e-04,9.176373e-06,1.064153e-04,1.650464e-06,0.000084,1.082588e-04,1.052501e-05,2.871225e-06,2.028176e-06,1.500093e-06,2.877328e-05,9.630749e-06,9.427003e-07,1.6

In [ ]:
import tensorflow as tf

precision_ncf = tf.keras.metrics.Precision(top_k=TOP_K)
recall_ncf = tf.keras.metrics.Recall(top_k=TOP_K)

precision_ncf.update_state(R_ij, pred)
recall_ncf.update_state(R_ij, pred)

In [ ]:
print(
    f"At K = {TOP_K}, we have a precision of {precision_ncf.result().numpy():.5f}
     and a recall of {recall_ncf.result().numpy():.5f}"
)

At K = 5, we have a precision of 0.75380 and a recall of 0.03078
